In [1]:
import pandas as pd
import os

from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModel

/home/nathalia/.pyenv/versions/3.10.6/envs/stocks_vs_behavior/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-25 18:33:10.355881: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-25 18:33:10.389055: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-25 18:33:11.053453: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-25 18:33:13.246315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to us

# Load Data

In [2]:
processed_data_path = os.path.join(
    os.path.abspath('../'),
    'data',
    'processed'
)

In [3]:
df_threads_text = pd.read_csv(
     os.path.join(
         processed_data_path,
        'customer_support_on_twitter_threads_text.csv'
    ),
    index_col=0
)

In [4]:
df_threads_text

,thread_id,n_tweets,ended_at,started_at,duration,thread_text
0,True,1,2011-08-29 03:20:05+00:00,2011-08-29 03:20:05+00:00,0,"Lost your booking number? No worries, just dro..."
1,2.0,4,2014-12-04 17:57:42+00:00,2014-12-04 00:22:00+00:00,1055,@119616 The .nbi files are part of the backup ...
2,3.0,1,2013-12-13 15:31:53+00:00,2013-12-13 15:31:53+00:00,0,Unreservable coaches with the new timetable ar...
3,4.0,2,2014-08-27 07:29:33+00:00,2014-08-27 06:31:06+00:00,58,"Hello @121643 , please provide the affected mo..."
4,5.0,2,2016-02-07 10:47:37+00:00,2016-02-07 10:40:43+00:00,6,@121880 The Premium subscription 21660 has bee...
...,...,...,...,...,...,...
423,424.0,6,2015-12-11 20:44:39+00:00,2015-12-11 20:38:22+00:00,6,@817936 If you put reason as same day refund a...
424,425.0,2,2016-04-15 10:15:53+00:00,2015-09-06 18:05:56+00:00,319209,"@818934 Hello Mary, we just reply to your DM. ..."
425,426.0,3,2014-12-09 17:05:05+00:00,2014-12-09 11:54:41+00:00,310,@822638 Frozen on message: Preparing to turn o...
426,427.0,2,2015-11-20 13:07:03+00:00,2015-11-20 08:15:51+00:00,291,"@822842 We'd like to look into this, Gordan. C..."


# Sentiment Analysis

## Load Model

In [5]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/nathalia/.pyenv/versions/3.10.6/envs/stocks_vs_behavior/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Check model

In [22]:
data_input = df_threads_text['thread_text'][423]
encoded_input = tokenizer(data_input, return_tensors='tf')

In [23]:
output = model(encoded_input)
scores = output.logits.numpy()[0]
predicted_label = int(scores.argmax())
predicted_label

0

In [24]:
scores

array([ 0.10046393, -0.06955207, -0.02814335], dtype=float32)

In [25]:
data_input

"@817936 If you put reason as same day refund and explain got wrong ticket, I'd expect the £10 will likely be waived. -Ollie @GWRHelp Yes, I bought them this afternoon and they're not for use until January. @817936 Did you buy the tickets today? -Ollie @GWRHelp Ok, and will I be charged the £10 admin fee? Haven't printed the tickets from the machine at the station yet. @817936 Hi Kimberley. You would need to apply for refund and then buy the appropriate ticket. -Ollie @GWRHelp Hi, booked some tickets on your website earlier but need to change one from day return to open return. How can I do this?"

## Apply Model

In [27]:
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
df_sentiments = pd.DataFrame()
for i in range(0, len(df_threads_text) + 1, 100):
    batch = df_threads_text.iloc[:i]
    batch['Sentiment'] = batch['thread_text'].apply(sentiment_task)

    batch.to_csv(
        os.path.join(
            os.path.abspath('../'),
            'data',
            'tmp',
            f'threads_sentiment_batch_{i-100}_{i}.csv'
        )
    )

    df_sentiments = pd.concat([df_sentiments, batch])

/tmp/ipykernel_426093/3361097826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['Sentiment'] = batch['thread_text'].apply(sentiment_task)


In [29]:
batch

,thread_id,n_tweets,ended_at,started_at,duration,thread_text,Sentiment
0,True,1,2011-08-29 03:20:05+00:00,2011-08-29 03:20:05+00:00,0,"Lost your booking number? No worries, just dro...","[{'label': 'negative', 'score': 0.386934578418..."
1,2.0,4,2014-12-04 17:57:42+00:00,2014-12-04 00:22:00+00:00,1055,@119616 The .nbi files are part of the backup ...,"[{'label': 'positive', 'score': 0.409872591495..."
2,3.0,1,2013-12-13 15:31:53+00:00,2013-12-13 15:31:53+00:00,0,Unreservable coaches with the new timetable ar...,"[{'label': 'negative', 'score': 0.362931340932..."
3,4.0,2,2014-08-27 07:29:33+00:00,2014-08-27 06:31:06+00:00,58,"Hello @121643 , please provide the affected mo...","[{'label': 'negative', 'score': 0.369610667228..."
4,5.0,2,2016-02-07 10:47:37+00:00,2016-02-07 10:40:43+00:00,6,@121880 The Premium subscription 21660 has bee...,"[{'label': 'negative', 'score': 0.380617201328..."
...,...,...,...,...,...,...,...
95,96.0,1,2014-09-08 22:08:47+00:00,2014-09-08 22:08:47+00:00,0,Attention all Daniel Bowens: Please get your e...,"[{'label': 'positive', 'score': 0.380338102579..."
96,97.0,5,2014-02-12 04:00:47+00:00,2013-12-17 15:01:42+00:00,81419,"@255040 Hi, Kindly submit your request via E-F...","[{'label': 'positive', 'score': 0.398043304681..."
97,98.0,3,2016-02-12 04:22:23+00:00,2013-07-08 13:53:33+00:00,1365988,Hello @256145 You could also empty your inbox ...,"[{'label': 'positive', 'score': 0.399836421012..."
98,99.0,1,2015-07-20 17:13:35+00:00,2015-07-20 17:13:35+00:00,0,As @115993 celebrates a new beginning as #PYPL...,"[{'label': 'positive', 'score': 0.401550859212..."


In [31]:
batch['Sentiment'][96]

[{'label': 'positive', 'score': 0.39804330468177795}]